## Trabajo practico wisconsin  Aprendisaje Aumtomatico 1 
### Alumno: Morales Lucas Andrés

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#importamos los modelos:
from sklearn import linear_model
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier

#importamos las librerias para evaluar 
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score 


### importamos los datos


In [2]:
df=pd.read_csv('Aprendizaje_breast-cancer-wisconsin.tab', delimiter='\t')
df

,Clump thickness,Unif_Cell_Size,Unif_Cell_Shape,Marginal_Adhesion,Single_Cell_Size,Bare_Nuclei,Bland_Chromatine,Normal_Nucleoli,Mitoses,type,Selected
0,9.67,9.96,9.27,2.40,9.49,7.93,7.90,0.33,0.06,malign,No
1,0.14,0.15,0.47,0.82,1.71,0.19,2.22,0.55,0.49,benign,No
2,0.61,0.49,0.99,0.19,1.98,0.27,2.07,0.71,0.28,benign,No
3,7.29,1.11,3.42,0.07,4.63,0.25,4.17,3.76,3.11,malign,No
4,6.03,2.47,1.64,9.15,4.48,9.86,4.43,3.58,3.73,malign,No
...,...,...,...,...,...,...,...,...,...,...,...
578,5.00,4.92,4.65,5.29,2.63,9.22,2.81,0.78,0.13,malign,No
579,4.66,1.91,2.22,0.28,5.07,9.36,4.67,0.76,0.90,malign,No
580,2.90,0.77,0.24,3.74,2.32,0.23,1.19,1.12,0.94,benign,No
581,4.72,0.04,0.46,2.62,1.39,0.50,0.70,0.86,0.92,benign,No


### Quitamos los datos innecesarios


In [3]:
df=df.drop('Selected', axis=1)
df.head()

,Clump thickness,Unif_Cell_Size,Unif_Cell_Shape,Marginal_Adhesion,Single_Cell_Size,Bare_Nuclei,Bland_Chromatine,Normal_Nucleoli,Mitoses,type
0,9.67,9.96,9.27,2.40,9.49,7.93,7.90,0.33,0.06,malign
1,0.14,0.15,0.47,0.82,1.71,0.19,2.22,0.55,0.49,benign
2,0.61,0.49,0.99,0.19,1.98,0.27,2.07,0.71,0.28,benign
3,7.29,1.11,3.42,0.07,4.63,0.25,4.17,3.76,3.11,malign
4,6.03,2.47,1.64,9.15,4.48,9.86,4.43,3.58,3.73,malign


### DIvidimos en Train y Test 


In [4]:
X=df.drop('type', axis=1)
Y=df['type']

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.25,random_state=123)

### ahora dividimos nuevamente para obtener los Validation train y Validation Test respectivamente

In [6]:
X_val_train, X_val_test, Y_val_train, Y_val_test = train_test_split(X_train,Y_train,test_size=0.25,random_state=123)

### Seleccion de modelos e hiperparámetros :

#### Regresion Logistica:

In [7]:
modeloRLog= linear_model.LogisticRegression(max_iter=2000, penalty='none', fit_intercept=True , random_state=123)
modeloRLog.fit(X_val_train,Y_val_train)
Y_val_pred=modeloRLog.predict(X_val_test)

#Evaluamos el Validation Test set
AC_RLog= accuracy_score(Y_val_test, Y_val_pred)
F1_RLog=f1_score(Y_val_test, Y_val_pred, average='weighted')

#mostramos los resultados:
print("Evaluación modelo de Regresión Logística en el Validation")
print("Accuracy: ", round(AC_RLog,3))
print("F1: ", round(F1_RLog,3))


Evaluación modelo de Regresión Logística en el Validation
Accuracy:  0.955
F1:  0.954


#### kNN:

In [8]:
k_valores=np.arange(1,6,1)# fui probando con distintos arranges con distintos k y a partir de k=1 siempre em daba lo mismo

In [16]:
k_max=0
F1kNN_max=0
ACkNN_max=0

for k in k_valores:
    modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
    modelokNN.fit(X_val_train, Y_val_train)
    y_val_pred=modelokNN.predict(X_val_test)
    ACkNN=accuracy_score(Y_val_test, Y_val_pred)
     F1kNN=f1_score(Y_val_test,Y_val_pred, average='macro')
    
    if F1kNN>F1kNN_max:
        k_max=k
        F1kNN_max=F1kNN
        ACkNN_max=ACkNN
        
    
    
print('Valor de k: ', k_max)
print("Accuracy: ", round(ACkNN_max,3))
print("F1: ", round(F1kNN_max,3))

Valor de k:  1
Accuracy:  0.936
F1:  0.933


#### Arbol :

In [10]:
profundidades=np.arange(1,201,1)

p_Max=0               # profundidad para la cual obtendermos el mejor F1
ACT_Max=0
F1T_Max=0

for p in profundidades: 
    modeloT=DecisionTreeClassifier(max_depth=p, random_state=123)        #Creamos el modelo con profundidad p
    modeloT.fit(X_val_train, Y_val_train)              # Lo entrenamos
    Y_val_pred = modeloT.predict(X_val_test)           # Pronosticamos los valores de y para los valores del X_test
    AC_T=accuracy_score(Y_val_test, Y_val_pred)
    F1_T=f1_score(Y_val_test,Y_val_pred, average='macro')
    
    if F1_T>F1T_Max:
        p_Max=p
        F1T_Max=F1_T
        ACT_Max=AC_T
    
print('Valor de :profundidad ', p_Max)
print("Accuracy: ", round(ACT_Max,3))
print("F1: ", round(F1T_Max,3))

Valor de :profundidad  5
Accuracy:  0.955
F1:  0.952


### comparamos los resultados de todos los modelos:

In [11]:
print("RLog -> F1: ", round(F1_RLog,3), '| AC_RLog: ', round(AC_RLog,3))
print("kNN -> F1: ", round(F1kNN_max,3), '| ACkNN: ', round(ACkNN_max,3),"| k= ",k_max)
print("Árbol -> F1: ",round(F1T_Max,3),'| ACT: ', round(ACT_Max,3), "| p= ",p_Max )

RLog -> F1:  0.954 | AC_RLog:  0.955
kNN -> F1:  0.952 | ACkNN:  0.955 | k=  1
Árbol -> F1:  0.952 | ACT:  0.955 | p=  5


## EVALUACION

In [12]:
modeloRLog= linear_model.LogisticRegression(max_iter=2000, penalty='none', fit_intercept=True , random_state=123)
modeloRLog.fit(X_train,Y_train)
Y_pred=modeloRLog.predict(X_test)

AC_RLog_Final= accuracy_score(Y_test, Y_pred)
F1_RLog_Final=f1_score(Y_test, Y_pred, average='weighted')
#mostramos los resultados:
print("Evaluación modelo de Regresión Logística en el Validation")
print("Accuracy: ", round(AC_RLog_Final,3))
print("F1: ", round(F1_RLog_Final,3))


Evaluación modelo de Regresión Logística en el Validation
Accuracy:  0.993
F1:  0.993


In [13]:
k=1
modelokNN = neighbors.KNeighborsClassifier(n_neighbors=k,weights='uniform')
modelokNN.fit(X_train, Y_train)
y_pred=modelokNN.predict(X_test)
ACkNN=accuracy_score(Y_test, Y_pred)
F1kNN=f1_score(Y_test,Y_pred, average='macro')

    
print('Valor de k: ', k)
print("Accuracy: ", round(ACkNN,3))
print("F1: ", round(F1kNN,3))

Valor de k:  1
Accuracy:  0.993
F1:  0.992


In [17]:
modelo_produccion=linear_model.LogisticRegression(max_iter=2000, penalty='none', fit_intercept=True , random_state=123)
modelo_produccion.fit(X,Y)

LogisticRegression(max_iter=2000, penalty='none', random_state=123)

In [19]:
import pickle 
nombre_archivo='tp_wisconsin.sav'

pickle.dump(modelo_produccion, open (nombre_archivo,'wb'))